In [1]:
!pip install -q trino

In [2]:
# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'lakekeeper' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://lakekeeper:8181/catalog"
TRINO_URI = "http://trino:8080"
WAREHOUSE = "irisa-ot"

# Create Trino Catalog

In [3]:
from trino.dbapi import connect

conn = connect(host=TRINO_URI, user="trino")

In [4]:
cur = conn.cursor()
cur.execute(
    f"""
    CREATE CATALOG lakekeeper USING iceberg
    WITH (
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = '{CATALOG_URL}',
        "iceberg.rest-catalog.warehouse" = '{WAREHOUSE}',
        "iceberg.rest-catalog.security" = 'NONE',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "s3.region"= 'dummy',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://minio:9000',
        "fs.native-s3.enabled" = 'true'
    )
"""
)

## Read and Write Tables

In [5]:
# Connect directly to demo catalog, so that we don't have to use it as a prefix
conn = connect(host=TRINO_URI, user="trino", catalog="lakekeeper")
cur = conn.cursor()

In [9]:
cur.execute("CREATE SCHEMA IF NOT EXISTS trino_namespace")

In [11]:
cur.execute(
    "CREATE TABLE IF NOT EXISTS trino_namespace.trino_table (my_ints INT, my_floats DOUBLE, strings VARCHAR) WITH (format='PARQUET')"
)
cur.execute(
    "INSERT INTO trino_namespace.trino_table VALUES (1, 1.0, 'a'), (2, 2.0, 'b')"
)

In [12]:
cur.execute("SELECT * FROM trino_namespace.trino_table").fetchall()

[[1, 1.0, 'a'], [1, 1.0, 'a'], [2, 2.0, 'b'], [2, 2.0, 'b']]

In [6]:
cur.execute("SELECT * FROM banking.source_transactions").fetchall()

[['CB-3FCF984F2570',
  'core_banking',
  datetime.datetime(2025, 7, 23, 13, 46, 40, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  Decimal('7312.77'),
  'ACC60435536',
  'deposit',
  'REF-4210957669',
  'pending',
  '"{""description"": ""Room sell more past.""',
  None,
  None],
 ['CB-B6A0939EDC29',
  'core_banking',
  datetime.datetime(2025, 7, 9, 6, 9, 40, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  Decimal('7003.31'),
  'ACC50743151',
  'refund',
  'REF-7619756144',
  'pending',
  '"{""description"": ""By doctor threat.""',
  None,
  None],
 ['CB-2EE63F6D4446',
  'core_banking',
  datetime.datetime(2025, 6, 26, 20, 12, 48, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  Decimal('4705.67'),
  'ACC2048893',
  'fee',
  'REF-446307111',
  'pending',
  '"{""description"": ""Group character long leader.""',
  None,
  None],
 ['CB-4F88C174F252',
  'core_banking',
  datetime.datetime(2025, 7, 11, 4, 56, tzinfo=zoneinfo.ZoneInfo(key='UTC')),
  Decimal('8249.22'),
  'ACC13326781',
  'transfer',
  'REF-970421968